In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# from chromedriver_autoinstaller import install as install_chromedriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import csv
import pandas as pd
import json

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# url = "https://store.kyobobook.co.kr/bestseller/store/seoul/001/00?page=1"
urls=["https://store.kyobobook.co.kr/bestseller/store/seoul/001/00?page=1", # 광화문
    "https://store.kyobobook.co.kr/bestseller/store/seoul/015/00?page=1", # 강남
    "https://store.kyobobook.co.kr/bestseller/store/seoul/072/00?page=1", # 서울대
    "https://store.kyobobook.co.kr/bestseller/store/suburb/005/00?page=1", # 부산
    "https://store.kyobobook.co.kr/bestseller/store/suburb/087/00?page=1", # 광주
    "https://store.kyobobook.co.kr/bestseller/store/suburb/002/00?page=1" # 대전
]

# target_seoul = ['광화문점', '강남점', '서울대점']
target_not_seoul= ['부산점', '광주상무점', '대전점']
target_location = ['광화문점', '강남점', '서울대점', '부산점', '광주상무점', '대전점']

column_name=['location','ranking','book_number','title','book_img','book_detail','author','publisher','price','score','num_of_review','genre'
]
collected=[]

for location, url in zip(target_location, urls):
    driver.get(url)
    print(location, url)
    wait = WebDriverWait(driver, 5)

    try:
        wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'li.mt-9'))
        )
    except TimeoutException:
        print(f"ERROR: {location} 베스트셀러 목록 로드에 실패했습니다. (5초 초과)")

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('li.mt-9')

    ranking=1
    # 장르, 작가, 가격, 리뷰수, 리뷰 점수
    for item in items:
        book_number = item.select_one("a.prod_link.relative.block")['href'].split('/')[-1].strip()
        book_detail = item.select_one("a.prod_link.relative.block")['href']
        title = item.select_one("a.prod_link.line-clamp-2").text.strip()
        book_img = item.select_one("a.prod_link.relative.block div img")['src']

        time.sleep(5)
        author = item.select_one("div.line-clamp-2.flex.overflow-hidden").text.split('·')[0].strip()
        publisher = item.select_one("div.line-clamp-2.flex.overflow-hidden").text.split('·')[1].strip()
        price_text = item.select_one("div.flex.flex-row.flex-wrap.place-items-center").text
        if '%' in price_text:
            price = price_text.split('%')[1].split('원')[0]
        else:
            price = price_text.split('원')[0]
        
        review_score = item.select_one("div.flex.flex-row.items-center.gap-1.fz-14").text.split('클로버 아이콘')[1].split('(')[0].strip()
        if review_score == '0':
            review_num = 0
        else:
            review_num = item.select_one("div.flex.flex-row.items-center.gap-1.fz-14").text.split('(')[1].split('개의')[0].strip()
        
        driver.get(book_detail)
        time.sleep(5)
        wait = WebDriverWait(driver, 10)
        target_xpath = '//*[@id="mainDiv"]/main/section[1]/div/ol/li[3]/a'

        try:
            a_element = wait.until(
                EC.presence_of_element_located((By.XPATH, target_xpath))
            )
            genre = a_element.text.strip()
        except Exception as e:
            print(f"요소를 찾는 데 실패했습니다: {e}")
        
        driver.back()
        time.sleep(5)
        
        if location[-1] == '점':
            location = location[:-1]
        if location == '광주상무':
            location = location[:2]
        
        
        # 체크
        print(location, ranking, title, genre, book_img)
        collected.append([location, ranking, book_number, title, book_img, book_detail, author, \
            publisher, price, review_score, review_num, genre])
        ranking += 1


    json_data=[]
    for each in collected:
        tojson = dict(zip(column_name, each))
        json_data.append(tojson)

    with open('location_bestseller'+location+'.json', 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)


광화문점 https://store.kyobobook.co.kr/bestseller/store/seoul/001/00?page=1
광화문 1 호의에 대하여 시/에세이 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791173323027.jpg
광화문 2 혼모노 소설 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9788936439743.jpg
광화문 3 렛뎀 이론 자기계발 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791162544327.jpg
광화문 4 다정한 사람이 이긴다 시/에세이 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791193262658.jpg
광화문 5 시대예보: 경량문명의 탄생 경제/경영 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791170613114.jpg
광화문 6 자몽살구클럽 소설 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791199305304.jpg
광화문 7 모순 소설 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9788998441012.jpg
광화문 8 양면의 조개껍데기 소설 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791168343108.jpg
광화문 9 안녕이라 그랬어 소설 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791141602376.jpg
광화문 10 료의 생각 없는 생각 시/에세이 https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791170403319.jpg
광화문 11 소년이 온다 소설 

#### 저장된 변수 확인용

In [ ]:
json_data=[]
for each in collected:
    tojson = dict(zip(column_name, each))
    json_data.append(tojson)
print(json_data)

with open('location_bestseller_전국.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

[{'location': '광화문', 'ranking': 1, 'book_number': 'S000217393638', 'title': '호의에 대하여', 'book_img': 'https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791173323027.jpg', 'book_detail': 'https://product.kyobobook.co.kr/detail/S000217393638', 'author': '문형배', 'publisher': '김영사', 'price': '16,920', 'score': '9.35', 'num_of_review': '304', 'genre': '시/에세이'}, {'location': '광화문', 'ranking': 2, 'book_number': 'S000216111714', 'title': '혼모노', 'book_img': 'https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9788936439743.jpg', 'book_detail': 'https://product.kyobobook.co.kr/detail/S000216111714', 'author': '성해나', 'publisher': '창비', 'price': '16,200', 'score': '9.25', 'num_of_review': '596', 'genre': '소설'}, {'location': '광화문', 'ranking': 3, 'book_number': 'S000217249954', 'title': '렛뎀 이론', 'book_img': 'https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791162544327.jpg', 'book_detail': 'https://product.kyobobook.co.kr/detail/S000217249954', 'author': '멜 로빈스', 'publisher': '비즈니스북스', 'pr

In [6]:
collected

[['광화문',
  1,
  'S000217393638',
  '호의에 대하여',
  'https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791173323027.jpg',
  'https://product.kyobobook.co.kr/detail/S000217393638',
  '문형배',
  '김영사',
  '16,920',
  '9.35',
  '304',
  '시/에세이'],
 ['광화문',
  2,
  'S000216111714',
  '혼모노',
  'https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9788936439743.jpg',
  'https://product.kyobobook.co.kr/detail/S000216111714',
  '성해나',
  '창비',
  '16,200',
  '9.25',
  '596',
  '소설'],
 ['광화문',
  3,
  'S000217249954',
  '렛뎀 이론',
  'https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791162544327.jpg',
  'https://product.kyobobook.co.kr/detail/S000217249954',
  '멜 로빈스',
  '비즈니스북스',
  '18,000',
  '9.59',
  '228',
  '자기계발'],
 ['광화문',
  4,
  'S000217241544',
  '다정한 사람이 이긴다',
  'https://contents.kyobobook.co.kr/sih/fit-in/300x0/pdt/9791193262658.jpg',
  'https://product.kyobobook.co.kr/detail/S000217241544',
  '이해인',
  '필름(Feelm)',
  '17,100',
  '9.7',
  '232',
  '시/에세이'],
 ['광화문',
  5,
  'S00021742195